In [ ]:
from traitlets import Unicode, Bool, validate, TraitError, observe
from ipywidgets import DOMWidget, register
import time
import threading
import json

In [ ]:
%%javascript

require.undef('email_widget');

define('email_widget', ["@jupyter-widgets/base"], function(widgets) {
    var WebVisualizerView = widgets.DOMWidgetView.extend({
        sleep: function(time_ms) {
            return new Promise((resolve) => setTimeout(resolve, time_ms));
        },
        
        jspy_send: function(message) {
            this.model.set("jspy_channel", message);
            this.touch();
        },
        
        // args must be all strings.
        // TODO: kwargs and sanity check
        callPython: async function(func, args=[]) {
            var message = {
                func: func,
                args: args
            };
            this.jspy_send(JSON.stringify(message));
            var count = 0;
            while (!this.new_pyjs_message) {
//                 await this.sleep(10);
                console.log("!!!!!!!!!!!!!!! Await ", count++)
                await new Promise(resolve => setTimeout(resolve, 10))
            }
            console.log("!!!!!!!!!!!!! Await done.")
            this.new_pyjs_message = false;
            var message = this.model.get('pyjs_channel');
            return message
        },
        
        callPythonWrapper: function(func, args=[]) {
            return callPython(func, args).then((result) => (result));
        },
        
        render: function() {
            this.new_pyjs_message = false;
            this.email_input = document.createElement('p');
            this.el.appendChild(this.email_input);
            
            // Listen for py->js message.
            this.model.on('change:pyjs_channel', this.on_pyjs_message, this);
            
            // Send js->py message for testing.
            this.callPython("call_http_request", 
                            ["my_entry_point", "my_query_string", "my_data"]).then(
                (result) => {console.log("callPython.then()", result)}
            );
        },
        
        on_pyjs_message: function() {
            var message = this.model.get('pyjs_channel');
            message = "pyjs_message received: " + message;
            this.email_input.innerText = this.email_input.innerText 
                                       + "\n" 
                                       + message;
            this.new_pyjs_message = true;
        },
        
    });
    
    return {
        WebVisualizerView: WebVisualizerView
    }
});

In [ ]:
def py2js_send_five_messages(widget):
    def func():
        for i in range(3):
            widget.pyjs_channel = f"Hello from Python: {i+1}"
            time.sleep(1)
    return func

jspy_message = ""

@register
class WebVisualizer(DOMWidget):
    _view_name = Unicode('WebVisualizerView').tag(sync=True)
    _view_module = Unicode('email_widget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    
    # Attributes
    pyjs_channel = Unicode("Empty pyjs_channel.", help="Python->JS message channel.").tag(sync=True)
    jspy_channel = Unicode("Empty jspy_channel.", help="JS->Python message channel.").tag(sync=True)
    
    def pyjs_send(self, message):
        self.pyjs_channel = message
        
    def call_http_request(self, entry_point, query_string, data):
        return f"Called Http Request: {entry_point}, {query_string}, {data}!"

    @observe('jspy_channel')
    def on_jspy_message(self, change):
        jspy_message = change["new"]
        print(f"jspy message: {jspy_message}")
        try:
            jspy_request = json.loads(jspy_message)
            if "func" not in jspy_request or jspy_request["func"] != "call_http_request":
                raise ValueError(f"Invalid jspy function: {jspy_message}")
            if "args" not in jspy_request or len(jspy_request["args"]) != 3:
                raise ValueError(f"Invalid jspy function arguments: {jspy_message}")
            result = self.call_http_request(jspy_request["args"][0], 
                                            jspy_request["args"][1], 
                                            jspy_request["args"][2])
            self.pyjs_send(result)
        except:
            print(f"jspy message is not a valid function call: {jspy_message}")

In [ ]:
visualizer = WebVisualizer()
visualizer